In [1]:
from hdfs import InsecureClient
from confluent_kafka import Consumer
import json

In [2]:
hdfs_client = InsecureClient('http://20.239.82.205:9870', user='haihp02')

In [9]:
hdfs_client.delete('/haihp02/real_estate_data/nhadatviet.jsonl')

True

In [4]:
local_jsonl_file = './guland-ha-noi-hai-ba-trung.jsonl'
hdfs_file_path = '/haihp02/real_estate_data/guland-ha-noi-hai-ba-trung.jsonl'
with open(local_jsonl_file, 'r') as local_file:
    jsonl_content = local_file.read()
with hdfs_client.write(hdfs_path=hdfs_file_path, encoding='utf-8', append=False) as writer:
    writer.write(jsonl_content)

In [5]:
with hdfs_client.read('/haihp02/real_estate_data/nhadatviet.jsonl', encoding='utf-8') as reader:
    content = reader.readlines()

In [7]:
# def json_to_jsonl(json_file_path, jsonl_file_path):
#     with open(json_file_path, 'r', encoding='utf8') as f:
#         data = json.load(f)
#     json_lines = [json.dumps(item, ensure_ascii=False) for item in data]
#     with open(jsonl_file_path, 'w', encoding='utf8') as f:
#         for line in json_lines:
#             f.write(f'{line}\n')

# json_file_path = '/home/haihp02/Edu/Big data storage and processing/data/nhadatviet.json'
# jsonl_file_path = '/home/haihp02/Edu/Big data storage and processing/data/nhadatviet.jsonl'
# json_to_jsonl(json_file_path, jsonl_file_path)

In [ ]:
def run_hdfs_consumer():
    c = Consumer({'bootstrap.servers': '0.0.0.0:9192,0.0.0.0:9292,0.0.0.0:9392',
                  'group.id': '123'})
    c.subscribe(['bds', 'i-batdongsan', 'nhadatviet'])
    print(c.assignment())
    consumed = []

    while True:
        msg = c.poll(0.01)

        if msg is None:
            print(len(consumed))
            continue
        if msg.error():
            print(f'Consumer error: {msg.error()}')
            continue
        
        msg_received = json.loads(msg.value().decode("utf-8"))
        consumed.append(msg_received)
        print(f'Received message in topic: {msg.topic()}')
        print(msg.key())
        print(msg.offset())

run_hdfs_consumer()